In [1]:
import os # operating system
import glob # find the path
import pandas as pd
import numpy as np
import cv2
from matplotlib import image
import PIL
import pickle
import tensorflow
import tensorflow.keras as k
import pandas_ml
import matplotlib.pyplot as plt
%matplotlib inline

## Load Data

In [2]:
data = np.load('data_with_pca.npz')
data.files

['arr_0', 'arr_1']

In [3]:
x = data['arr_0']
y = data['arr_1']
x.shape,y.shape        

((138, 64, 64), (138,))

In [4]:
x =x.reshape((138, 64, 64,1))
x.shape

(138, 64, 64, 1)

In [5]:
x = x/255 # Normalize

In [6]:
from sklearn.preprocessing import OneHotEncoder

In [7]:
ohe = OneHotEncoder()
y = y.reshape(-1,1)
y_ohe = ohe.fit_transform(y).toarray()
y_ohe.shape

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


(138, 2)

In [8]:
y_ohe

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.

In [9]:
from sklearn.model_selection import train_test_split

In [10]:
x_train,x_test,y_train,y_test = train_test_split(x,y_ohe,test_size=0.2)

In [11]:
x_train.shape,x_test.shape,y_train.shape,y_test.shape

((110, 64, 64, 1), (28, 64, 64, 1), (110, 2), (28, 2))

**Buliding Convolutional Neural Network**
- **input(none(it's just the num),64,64,1)**
- **Convolution(3X3)(30)-Stride=1 - act-relu(rectified linear unit) -Output - ((N-F)/S+1)(con is possible if this integer)-(N,62,62,30)**
- **Convolution(5X5)(60)(only it should be odd)-Stride=1 - act-relu- Output-(N,58,58,60)**
- **Max Pooling(2X2)-Stride-2 -Output-(N,29,29,60)**
- **Convolution(3X3)(32)-Stride=1 - act-relu- Output-(N,27,27,32)**
- **Max Pooling(2X2)-Stride-2 -Output-(N,13,13,32)**
- **This Output is Input to Neural Network**
- **5408->256 Neurons -ac(relu)->128 Neurons-ac(relu) -> 2Neurons ac(sigmoid)**

In [12]:
from tensorflow.keras import Sequential, regularizers
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten
from  tensorflow.keras import callbacks

In [13]:
# CNN without regularization
def cnn():
    model = Sequential()
    model.add(Conv2D(filters=30,kernel_size=(3,3),strides=(1,1),activation='relu',input_shape=[64,64,1]))
    model.add(Conv2D(filters=60,kernel_size=(5,5),strides=(1,1),activation='relu'))
    model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
    model.add(Conv2D(filters=32,kernel_size=(3,3),strides=(1,1),activation='relu'))
    model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
    # Neural Network
    model.add(Flatten())
    model.add(Dense(units=256,activation='relu'))
    model.add(Dense(units=128,activation='relu'))
    model.add(Dense(units=2,activation='sigmoid'))
    return model

In [14]:
model_wr = cnn()

Instructions for updating:
Colocations handled automatically by placer.


In [15]:
model_wr.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 62, 62, 30)        300       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 58, 58, 60)        45060     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 29, 29, 60)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 27, 27, 32)        17312     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 5408)              0         
_________________________________________________________________
dense (Dense)                (None, 256)               1384704   
__________

In [16]:
model_wr.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [27]:
model_wr.fit(x=x_train,y=y_train,epochs=20,validation_data=[x_test,y_test],batch_size=30)

Train on 110 samples, validate on 28 samples
Epoch 1/20
110/110 [==============================] - 2s 17ms/sample - loss: 0.0461 - acc: 0.9909 - val_loss: 0.4577 - val_acc: 0.8929
Epoch 2/20
110/110 [==============================] - 2s 20ms/sample - loss: 0.0501 - acc: 0.9818 - val_loss: 0.1952 - val_acc: 0.8929
Epoch 3/20
110/110 [==============================] - 2s 23ms/sample - loss: 0.0442 - acc: 0.9818 - val_loss: 0.5952 - val_acc: 0.8571
Epoch 4/20
110/110 [==============================] - 2s 21ms/sample - loss: 0.0477 - acc: 0.9818 - val_loss: 0.0847 - val_acc: 0.9643
Epoch 5/20
110/110 [==============================] - 2s 21ms/sample - loss: 0.0248 - acc: 1.0000 - val_loss: 0.2041 - val_acc: 0.8929
Epoch 6/20
110/110 [==============================] - 2s 23ms/sample - loss: 0.0172 - acc: 0.9909 - val_loss: 0.0628 - val_acc: 1.0000
Epoch 7/20
110/110 [==============================] - 3s 24ms/sample - loss: 0.0059 - acc: 1.0000 - val_loss: 0.0619 - val_acc: 0.9643
Epoch 8/20

In [28]:
# when you bulid a data bulid with pca and without pca it doesn't improper accuracy it remove noise, feature extraction 

In [29]:
model_wr.save('face_rec_gender.h5')

In [30]:
y_pred = model_wr.predict_classes(x_test)
y_pred

array([1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1,
       1, 1, 0, 0, 1, 1], dtype=int64)

In [31]:
y_test

array([[0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.]])

In [32]:
y_true = y_test[:,1]

In [33]:
cm = pandas_ml.ConfusionMatrix(y_true,y_pred)
cm

Predicted  False  True  __all__
Actual                         
False         13     0       13
True           1    14       15
__all__       14    14       28

In [34]:
cm.classification_report

,precision,recall,F1_score,support
Classes,,,,
False,0.928571,1,0.962963,13
True,1,0.933333,0.965517,15
__avg / total__,0.966837,0.964286,0.964331,28
